## Memory


In [84]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import Annotated
from typing_extensions import TypedDict
from operator import add


class State(TypedDict):
    foo: str
    bar: Annotated[list[str], add]


def node_a(state: State):
    return {"foo": "a", "bar": ["a"]}


def node_b(state: State):
    return {"foo": "b", "bar": ["b"]}


workflow = StateGraph(State)
workflow.add_node(node_a)
workflow.add_node(node_b)
workflow.add_edge(START, "node_a")
workflow.add_edge("node_a", "node_b")
workflow.add_edge("node_b", END)

checkpointer = InMemorySaver()
graph = workflow.compile(checkpointer=checkpointer)
config = {"configurable": {"thread_id": "1"}}
graph.invoke({"foo": ""}, config)  # type: ignore

{'foo': 'b', 'bar': ['a', 'b']}

In [85]:
config = {"configurable": {"thread_id": "1"}}
list(graph.get_state_history(config))  # type: ignore

[StateSnapshot(values={'foo': 'b', 'bar': ['a', 'b']}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f077930-08bb-6295-8002-93bd955e08fc'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}, 'thread_id': '1'}, created_at='2025-08-12T15:42:50.153814+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f077930-08b2-6b3e-8001-e2da8fd0a554'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'foo': 'a', 'bar': ['a']}, next=('node_b',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f077930-08b2-6b3e-8001-e2da8fd0a554'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}, 'thread_id': '1'}, created_at='2025-08-12T15:42:50.150354+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f077930-08ab-6646-8000-f138d992f45d'}}, tasks=(PregelTask(id='a45c4c80-070e-11ef-c870-bfa80379db40', name='node_b', path=('__pregel_pull

In [86]:
user_id = "1"
namespace_for_memory = (user_id, "memories")

## Question Extraction


In [ ]:
from pydantic import BaseModel, Field
from typing import Literal, List, Optional, Union


class ParamsBase(BaseModel):
    name: str
    value: Union[int, float, str]
    units: Optional[str] = None

    def format_name(self) -> str:
        self.name = self.name.lower().strip().replace(" ", "_")
        return self.name

    def format_expected(self) -> str:
        formatted = "Value Name:" + self.format_name() + " Value: " + str(self.value)
        if self.units:
            unit_str = f" Units: {self.units}"
            formatted += unit_str
        return formatted


class Question(BaseModel):
    original_question: str
    params: Optional[List[ParamsBase]] = Field(
        description="A parameter found in the question text"
    )
    correct_answer: Optional[List[ParamsBase]]


from langchain.chat_models import init_chat_model
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

model = init_chat_model("gpt-5-mini", model_provider="openai")
structured_llm = model.with_structured_output(Question)

template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
    You are an assistant that only performs extraction. Do not solve, infer, or compute.

Task:
1) Return the original question text.
2) Extract every value mentioned in the question and label each as either:
   - "parameter": a value that appears directly in the question text.
   - "correct_answer": only if the question explicitly states the answer (e.g., "Answer: 42", "Correct answer is 7", "Expected output: 3.14").

Definition of “parameter” (must satisfy all):
- The value is present verbatim in the question text (not derived by calculation).
- It may be a number (integers, decimals, scientific notation), a constant (e.g., π), a symbol with an assigned value (e.g., g = 9.81 m/s^2), a boolean, a categorical/string choice, a range (e.g., 5–10), or a list of given values.
- Keep units and symbols with the value when provided (e.g., "2 kg", "30°", "9.81 m/s^2").
- If both a symbol and value are shown (e.g., "mass m = 2 kg"), record name="m", value="2", units="kg" (also keep a raw field if helpful).

Do NOT mark as parameters:
- Values you can only obtain by calculation or domain knowledge.
- Purely named variables without assigned values (e.g., "let m be mass").
- Hints, defaults, or typical constants that are not explicitly provided in the text.
- Information present only in an external figure/image unless described in text.

Output JSON (only this object, no extra text):
{
  "question_text": "<original question>",
  "parameters": [
    { "name": "<string|null>", "value": "<string>", "units": "<string|null>", "raw": "<verbatim snippet>" }
  ],
  "correct_answer": { "value": "<string>", "units": "<string|null>", "raw": "<verbatim snippet>" } | null
}

Examples:

Example 1 — Parameters only
Input:
"A car travels at speed v = 20 m/s for time t = 5 s. Compute the distance traveled."
Output:
{
  "question_text": "A car travels at speed v = 20 m/s for time t = 5 s. Compute the distance traveled.",
  "parameters": [
    { "name": "v", "value": "20", "units": "m/s", "raw": "v = 20 m/s" },
    { "name": "t", "value": "5",  "units": "s",    "raw": "t = 5 s" }
  ],
  "correct_answer": null
}

Example 2 — Explicit correct_answer
Input:
"What is the capital of France? Answer: Paris."
Output:
{
  "question_text": "What is the capital of France? Answer: Paris.",
  "parameters": [],
  "correct_answer": { "value": "Paris", "units": null, "raw": "Answer: Paris." }
}

Example 3 — Symbols and units
Input:
"A block of mass m = 2 kg is on an incline of 30°. The friction coefficient is μ = 0.2. Use g = 9.81 m/s^2. Find the acceleration."
Output:
{
  "question_text": "A block of mass m = 2 kg is on an incline of 30°. The friction coefficient is μ = 0.2. Use g = 9.81 m/s^2. Find the acceleration.",
  "parameters": [
    { "name": "m", "value": "2",     "units": "kg",      "raw": "m = 2 kg" },
    { "name": incline, "value": "30",   "units": "°",       "raw": "incline of 30°" },
    { "name": "μ", "value": "0.2",   "units": null,      "raw": "μ = 0.2" },
    { "name": "g", "value": "9.81",  "units": "m/s^2",   "raw": "g = 9.81 m/s^2" }
  ],
  "correct_answer": null
}
""",
        ),
        ("human", "{question}"),
    ]
)
prompt = """
Question:Two teenagers are pulling on ropes attached to a tree. The angle between the ropes is  30.0°. David pulls with a force of 400.0 N and Stephanie pulls with a force of 300.0 N. (a) Find the component form of the net force. (b) Find the magnitude of the resultant (net) force on the tree and the angle it makes with David’s rope.
 
 The correct answer is 
 a) Fnet = 660.0i + 150.0 j
 b) 676.6 N at theta =12.8 degress from Davids Rope
"""

result = structured_llm.invoke(prompt)

In [ ]:
def format_question_params(question: Question):
    lines = []

    for param in question.params or []:
        result = param.format_expected()
        if result is not None:
            lines.append(str(result + f" Value Type: Question Parameter"))

    for ans in question.correct_answer or []:
        result = ans.format_expected()
        if result is not None:
            lines.append(str(result +  f" Value Type: Correct Answer") )

    return "\n\n".join(lines)


val = format_question_params(result)
print(val)

Value Name:angle_between_ropes Value: 30.0 Units: degrees Value Type: Question Parameter

Value Name:david_force Value: 400.0 Units: N Value Type: Question Parameter

Value Name:stephanie_force Value: 300.0 Units: N Value Type: Question Parameter

Value Name:fnet_x Value: 660.0 Units: N Value Type: Correct Answer

Value Name:fnet_y Value: 150.0 Units: N Value Type: Correct Answer

Value Name:resultant_magnitude Value: 676.6 Units: N Value Type: Correct Answer

Value Name:theta_from_david Value: 12.8 Units: degrees Value Type: Correct Answer
